# 📊 Análisis de Contratos Post-Electorales
## Versión Google Colab

**Hipótesis:** Existe un aumento en la cantidad de contratos después de las elecciones debido a que estos proveedores eran donantes de los partidos políticos.

---

## 1️⃣ Instalación y Setup

In [ ]:
# Importar librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 2️⃣ Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive montado correctamente")

Mounted at /content/drive
✅ Google Drive montado correctamente


## 3️⃣ Cargar Sistema de Análisis

In [ ]:
import os

folder = "/content/drive/MyDrive/Concurso/Contratos"
try:
    if not os.path.isdir(folder):
        raise FileNotFoundError(f"Folder not found: {folder}")

    all_files = [
        os.path.join(folder, f)
        for f in os.listdir(folder)
        if f.lower().endswith('.xlsx')
    ]

    dataframes = []
    for file in all_files:
        try:
            df = pd.read_excel(file, sheet_name='Informacion de contratos')
            dataframes.append(df)
        except Exception as e:
            print(f"Skipping file {file} due to error: {e}")

    if not dataframes:
        raise ValueError("No valid files could be loaded.")

    contratos = pd.concat(dataframes, ignore_index=True)
    contratos.head()
except Exception as e:
    print(f"Error: {e}")

print("✅ Sistema de análisis cargado")

✅ Sistema de análisis cargado


## 4️⃣ Cargar Datos (Tu código existente adaptado)

In [ ]:
# CONFIGURACIÓN - Ajusta estas rutas según tu estructura de Drive
RUTA_APORTACIONES = '/content/drive/MyDrive/Concurso/acumulado.xlsx'
CARPETA_CONTRATOS = '/content/drive/MyDrive/Concurso/Contratos'

# Cargar aportaciones
print("📂 Cargando aportaciones...")
aportaciones = pd.read_excel(RUTA_APORTACIONES, sheet_name='BBDD')
print(f"✅ {len(aportaciones):,} registros de aportaciones cargados")
print(f"   Columnas: {list(aportaciones.columns)}")

# Cargar contratos desde múltiples archivos
print(f"\n📂 Cargando contratos desde: {CARPETA_CONTRATOS}")
import os

try:
    if not os.path.isdir(CARPETA_CONTRATOS):
        raise FileNotFoundError(f"Carpeta no encontrada: {CARPETA_CONTRATOS}")

    all_files = [
        os.path.join(CARPETA_CONTRATOS, f)
        for f in os.listdir(CARPETA_CONTRATOS)
        if f.lower().endswith('.xlsx')
    ]

    print(f"   Archivos encontrados: {len(all_files)}")

    dataframes = []
    for file in all_files:
        try:
            df = pd.read_excel(file, sheet_name='Informacion de contratos')
            dataframes.append(df)
            print(f"   ✓ {os.path.basename(file)}: {len(df):,} contratos")
        except Exception as e:
            print(f"   ✗ Error en {os.path.basename(file)}: {e}")

    if not dataframes:
        raise ValueError("No se pudieron cargar archivos válidos.")

    contratos = pd.concat(dataframes, ignore_index=True)
    print(f"\n✅ Total: {len(contratos):,} contratos cargados")
    print(f"   Columnas: {list(contratos.columns[:5])}...")  # Mostrar primeras 5

except Exception as e:
    print(f"❌ Error: {e}")

📂 Cargando aportaciones...
✅ 97,010 registros de aportaciones cargados
   Columnas: ['TIPO CONTRIBUCIÓN', 'TIPO PERSONA', 'PARTIDO POLÍTICO', 'FECHA', 'CÉDULA', 'NOMBRE DEL CONTRIBUYENTE', 'MONTO']

📂 Cargando contratos desde: /content/drive/MyDrive/Concurso/Contratos
   Archivos encontrados: 31
   ✓ 072021-122021.xlsx: 30,937 contratos
   ✓ 072023-122023.xlsx: 36,935 contratos
   ✓ 072024-122024.xlsx: 40,995 contratos
   ✓ 012025-062025.xlsx: 28,663 contratos
   ✓ 012021-062021.xlsx: 70 contratos
   ✓ 012024-062024.xlsx: 24,658 contratos
   ✓ 012023-062023.xlsx: 20,555 contratos
   ✓ 012022-062022.xlsx: 19,861 contratos
   ✓ 012016-062016.xlsx: 4,640 contratos
   ✓ 072016-122016.xlsx: 11,053 contratos
   ✓ 012017-062017.xlsx: 7,933 contratos
   ✓ 012018-062018.xlsx: 9,435 contratos
   ✓ 072018-122018.xlsx: 18,347 contratos
   ✓ 012019-062019.xlsx: 11,961 contratos
   ✓ 072019-122019.xlsx: 23,997 contratos
   ✓ 072020-122020.xlsx: 22,882 contratos
   ✓ 012010-062010.xlsx: 0 contratos
 

In [ ]:
"""
Sistema de Análisis de Contratos Post-Electorales - Versión Colab V2 COMPLETA
Mejoras: Detección automática de columnas, filtro por partido, figsize=(10,5) por defecto
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
from matplotlib.dates import YearLocator, DateFormatter
from datetime import datetime, timedelta
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


class AnalizadorContratosElectorales:
    """
    Clase para analizar la relación temporal entre donaciones políticas y contratos gubernamentales
    Versión mejorada con detección flexible de columnas
    """

    ELECCIONES_CR = [
        {'fecha': '2010-02-07', 'partido_ganador': 'LIBERACION NACIONAL', 'presidente': 'Laura Chinchilla'},
        {'fecha': '2014-02-02', 'partido_ganador': 'ACCION CIUDADANA', 'presidente': 'Luis Guillermo Solís'},
        {'fecha': '2018-04-01', 'partido_ganador': 'ACCION CIUDADANA', 'presidente': 'Carlos Alvarado'},
        {'fecha': '2022-04-03', 'partido_ganador': 'PROGRESO SOCIAL DEMOCRATICO', 'presidente': 'Rodrigo Chaves'}
    ]

    COLORES_PARTIDOS = {
        'LIBERACION NACIONAL': ['#0B5D1E', '#90EE90'],
        'ACCION CIUDADANA': ['#FFD700', '#FFA500'],
        'FRENTE AMPLIO': ['#DC143C', '#FF6B6B'],
        'UNIDAD SOCIAL CRISTIANA': ['#4169E1', '#87CEEB'],
        'MOVIMIENTO LIBERTARIO': ['#8B008B', '#DDA0DD'],
        'RESTAURACION NACIONAL': ['#00CED1', '#AFEEEE'],
        'PROGRESO SOCIAL DEMOCRATICO': ['#FF4500', '#FFB347'],
    }

    def __init__(self, df_contratos, df_aportaciones):
        """Inicializa el analizador con DataFrames ya cargados"""
        print("📊 Preparando datos para análisis...")
        print(f"\n🔍 Analizando estructura de datos...")

        print(f"\nContratos - columnas encontradas:")
        for i, col in enumerate(df_contratos.columns[:10], 1):
            print(f"  {i}. {col}")
        if len(df_contratos.columns) > 10:
            print(f"  ... y {len(df_contratos.columns) - 10} más")

        print(f"\nAportaciones - columnas encontradas:")
        for i, col in enumerate(df_aportaciones.columns[:10], 1):
            print(f"  {i}. {col}")
        if len(df_aportaciones.columns) > 10:
            print(f"  ... y {len(df_aportaciones.columns) - 10} más")

        self.df_contratos = self._preparar_contratos(df_contratos.copy())
        self.df_donaciones = self._preparar_donaciones(df_aportaciones.copy())
        self.df_elecciones = self._crear_df_elecciones()

        print(f"\n✅ Contratos procesados: {len(self.df_contratos):,}")
        print(f"✅ Donaciones procesadas: {len(self.df_donaciones):,}")
        print(f"✅ Proveedores únicos: {self.df_contratos['cedula_proveedor'].nunique():,}")
        print(f"✅ Donantes únicos: {self.df_donaciones['cedula'].nunique():,}")

        self._identificar_coincidencias()

    def _encontrar_columna(self, df, palabras_clave, nombre_busqueda="columna"):
        """
        Busca una columna que contenga alguna de las palabras clave

        Parámetros:
        -----------
        df : DataFrame
        palabras_clave : list, lista de listas de palabras que deben estar presentes
        nombre_busqueda : str, nombre descriptivo para mensajes

        Retorna:
        --------
        str, nombre de la columna encontrada o None
        """
        columnas = df.columns.tolist()

        for palabras in palabras_clave:
            for col in columnas:
                col_lower = col.lower()
                # Verificar que todas las palabras estén presentes
                if all(palabra.lower() in col_lower for palabra in palabras):
                    print(f"  ✓ {nombre_busqueda}: '{col}'")
                    return col

        return None

    def _preparar_contratos(self, df):
        """Prepara y normaliza datos de contratos con detección mejorada"""
        print("\n🔧 Preparando contratos...")

        # Buscar columna de fecha
        fecha_col = self._encontrar_columna(
            df,
            [['fecha', 'notif'], ['fecha'], ['date']],
            "Fecha"
        )

        if not fecha_col:
            raise ValueError(
                "❌ No se encontró columna de fecha. "
                f"Columnas disponibles: {df.columns.tolist()}"
            )

        # Buscar columna de cédula del proveedor
        cedula_col = self._encontrar_columna(
            df,
            [
                ['cédula', 'proveedor'],
                ['cedula', 'proveedor'],
                ['cédula'],
                ['cedula'],
                ['identificación', 'proveedor'],
                ['identificacion', 'proveedor'],
                ['id', 'proveedor']
            ],
            "Cédula Proveedor"
        )

        if not cedula_col:
            # Intentar encontrar cualquier columna con cédula o identificación
            posibles = [col for col in df.columns
                       if any(x in col.lower() for x in ['cédula', 'cedula', 'identificación', 'identificacion', 'id'])]

            if posibles:
                print(f"  ⚠️  Columnas posibles encontradas: {posibles}")
                print(f"  → Usando: '{posibles[0]}'")
                cedula_col = posibles[0]
            else:
                raise ValueError(
                    f"❌ No se encontró columna de cédula/identificación del proveedor.\n"
                    f"Columnas disponibles: {df.columns.tolist()}\n"
                    f"Por favor, especifica manualmente: df_contratos.rename(columns={{'TU_COLUMNA': 'cedula_proveedor'}})"
                )

        # Buscar otras columnas opcionales
        identificador_col = self._encontrar_columna(
            df,
            [['identificador'], ['id', 'contrato'], ['numero', 'contrato'], ['nro', 'contrato']],
            "Identificador"
        )

        tipo_mod_col = self._encontrar_columna(
            df,
            [['tipo', 'modif'], ['modificacion']],
            "Tipo Modificación"
        )

        # Renombrar columnas
        mapeo = {
            fecha_col: 'fecha_notificación',
            cedula_col: 'cedula_proveedor'
        }

        if identificador_col:
            mapeo[identificador_col] = 'identificador'

        if tipo_mod_col:
            mapeo[tipo_mod_col] = 'tipo_modificacion'

        df = df.rename(columns=mapeo)

        # Convertir fecha
        df['fecha_notificación'] = pd.to_datetime(df['fecha_notificación'], errors='coerce')

        # Normalizar cédula
        df['cedula_proveedor'] = df['cedula_proveedor'].astype(str).str.strip()

        # Eliminar filas con fecha inválida
        antes = len(df)
        df = df.dropna(subset=['fecha_notificación'])
        if len(df) < antes:
            print(f"  ⚠️  Eliminadas {antes - len(df)} filas con fecha inválida")

        return df

    def _preparar_donaciones(self, df):
        """Prepara y normaliza datos de donaciones con detección mejorada"""
        print("\n🔧 Preparando donaciones...")

        # Buscar columna de partido
        partido_col = self._encontrar_columna(
            df,
            [
                ['partido', 'político'],
                ['partido', 'politico'],
                ['partido'],
                ['party']
            ],
            "Partido"
        )

        if not partido_col:
            raise ValueError(
                f"❌ No se encontró columna de partido político.\n"
                f"Columnas disponibles: {df.columns.tolist()}"
            )

        # Buscar columna de fecha
        fecha_col = self._encontrar_columna(
            df,
            [['fecha'], ['date'], ['fecha', 'donación'], ['fecha', 'aportación']],
            "Fecha"
        )

        if not fecha_col:
            raise ValueError(
                f"❌ No se encontró columna de fecha.\n"
                f"Columnas disponibles: {df.columns.tolist()}"
            )

        # Buscar columna de cédula
        cedula_col = self._encontrar_columna(
            df,
            [['cédula'], ['cedula'], ['identificación'], ['identificacion'], ['id']],
            "Cédula"
        )

        if not cedula_col:
            raise ValueError(
                f"❌ No se encontró columna de cédula.\n"
                f"Columnas disponibles: {df.columns.tolist()}"
            )

        # Buscar columna de monto
        monto_col = self._encontrar_columna(
            df,
            [['monto'], ['valor'], ['amount'], ['cantidad']],
            "Monto"
        )

        # Buscar columna de nombre
        nombre_col = self._encontrar_columna(
            df,
            [['nombre', 'contribuyente'], ['nombre'], ['donante'], ['aportante']],
            "Nombre"
        )

        # Renombrar columnas
        mapeo = {
            partido_col: 'partido_político',
            fecha_col: 'fecha',
            cedula_col: 'cedula'
        }

        if monto_col:
            mapeo[monto_col] = 'monto'

        if nombre_col:
            mapeo[nombre_col] = 'nombre_del_contribuyente'

        df = df.rename(columns=mapeo)

        # Convertir fecha
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

        # Normalizar cédula (remover guiones)
        df['cedula'] = df['cedula'].astype(str).str.replace('-', '').str.strip()

        # Eliminar filas con fecha inválida
        antes = len(df)
        df = df.dropna(subset=['fecha'])
        if len(df) < antes:
            print(f"  ⚠️  Eliminadas {antes - len(df)} filas con fecha inválida")

        return df

    def _crear_df_elecciones(self):
        """Crea DataFrame con información de elecciones"""
        df = pd.DataFrame(self.ELECCIONES_CR)
        df['fecha'] = pd.to_datetime(df['fecha'])
        return df

    def _identificar_coincidencias(self):
        """Identifica cédulas que aparecen como donantes Y proveedores"""
        cedulas_contratos = set(self.df_contratos['cedula_proveedor'].unique())
        cedulas_donaciones = set(self.df_donaciones['cedula'].unique())

        self.coincidencias = cedulas_contratos & cedulas_donaciones

        print(f"\n🔍 Encontradas {len(self.coincidencias)} cédulas que son DONANTES y PROVEEDORES")

    def detectar_alertas_temporales(self, ventana_meses=6, solo_coincidencias=True):
        """Detecta casos donde una donación y un contrato ocurren en menos de X meses"""
        print(f"\n🔎 Detectando alertas temporales (ventana: {ventana_meses} meses)...")

        alertas = []

        if solo_coincidencias:
            cedulas_analizar = self.coincidencias
        else:
            cedulas_analizar = set(self.df_donaciones['cedula'].unique())

        for cedula in cedulas_analizar:
            donaciones = self.df_donaciones[self.df_donaciones['cedula'] == cedula]
            contratos = self.df_contratos[self.df_contratos['cedula_proveedor'] == cedula]

            if len(contratos) == 0:
                continue

            for _, donacion in donaciones.iterrows():
                for _, contrato in contratos.iterrows():
                    diferencia_dias = abs((contrato['fecha_notificación'] - donacion['fecha']).days)
                    diferencia_meses = diferencia_dias / 30.44

                    if diferencia_meses <= ventana_meses:
                        alerta = {
                            'cedula': cedula,
                            'fecha_donacion': donacion['fecha'],
                            'fecha_contrato': contrato['fecha_notificación'],
                            'partido_donado': donacion.get('partido_político', 'N/A'),
                            'monto_donacion': donacion.get('monto', 0),
                            'diferencia_dias': diferencia_dias,
                            'diferencia_meses': diferencia_meses,
                            'donacion_antes': donacion['fecha'] < contrato['fecha_notificación'],
                            'año_donacion': donacion['fecha'].year,
                            'año_contrato': contrato['fecha_notificación'].year
                        }

                        if 'nombre_del_contribuyente' in donacion:
                            alerta['nombre_contribuyente'] = donacion['nombre_del_contribuyente']

                        if 'identificador' in contrato:
                            alerta['identificador_contrato'] = contrato['identificador']

                        if 'tipo_modificacion' in contrato:
                            alerta['tipo_modificacion'] = contrato['tipo_modificacion']

                        alertas.append(alerta)

        df_alertas = pd.DataFrame(alertas)

        if len(df_alertas) > 0:
            print(f"⚠️  Total de alertas detectadas: {len(df_alertas)}")
        else:
            print("✅ No se detectaron alertas en la ventana especificada")

        return df_alertas

    def visualizar_linea_temporal_partido(self, partido, fecha_inicio=None, fecha_fin=None,
                                          figsize=(10, 5), mostrar_alertas=True,
                                          ventana_alerta_meses=6):
        """Crea visualización tipo dashboard para un partido político específico"""
        partido_upper = partido.upper()

        donaciones_partido = self.df_donaciones[
            self.df_donaciones['partido_político'].str.upper() == partido_upper
        ]
        cedulas_partido = set(donaciones_partido['cedula'].unique())

        contratos_donantes = self.df_contratos[
            self.df_contratos['cedula_proveedor'].isin(cedulas_partido)
        ].copy()

        if fecha_inicio:
            contratos_donantes = contratos_donantes[
                contratos_donantes['fecha_notificación'] >= fecha_inicio
            ]
        if fecha_fin:
            contratos_donantes = contratos_donantes[
                contratos_donantes['fecha_notificación'] <= fecha_fin
            ]

        if len(contratos_donantes) == 0:
            print(f"⚠️  No hay contratos de donantes de {partido} en el período seleccionado")
            return None, None

        contratos_mensuales = contratos_donantes.groupby(
            pd.Grouper(key='fecha_notificación', freq='M')
        ).size()

        if fecha_inicio and fecha_fin:
            idx_completo = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='M')
            contratos_mensuales = contratos_mensuales.reindex(idx_completo, fill_value=0)

        alertas = None
        if mostrar_alertas:
            alertas_todas = self.detectar_alertas_temporales(ventana_alerta_meses)
            if len(alertas_todas) > 0 and 'partido_donado' in alertas_todas.columns:
                alertas = alertas_todas[
                    alertas_todas['partido_donado'].str.upper() == partido_upper
                ]

        fig, ax = plt.subplots(figsize=figsize)

        colores = self.COLORES_PARTIDOS.get(partido_upper, ['#4A90E2', '#E94F9E'])

        x = contratos_mensuales.index
        y = contratos_mensuales.values

        y1 = y * 0.65
        y2 = y * 0.35

        width = 20
        ax.bar(x, y1, color=colores[0], alpha=0.85, width=width)
        ax.bar(x, y2, bottom=y1, color=colores[1], alpha=0.85, width=width)

        if len(x) > 3:
            x_numeric = np.arange(len(x))
            mask = y > 0
            if mask.sum() > 2:
                z = np.polyfit(x_numeric[mask], y[mask], min(2, mask.sum()-1))
                p = np.poly1d(z)
                y_trend = p(x_numeric)
                ax.plot(x, y_trend, color='red', linewidth=4, zorder=5, alpha=0.8)

        elecciones_periodo = self.df_elecciones.copy()
        if fecha_inicio:
            elecciones_periodo = elecciones_periodo[elecciones_periodo['fecha'] >= fecha_inicio]
        if fecha_fin:
            elecciones_periodo = elecciones_periodo[elecciones_periodo['fecha'] <= fecha_fin]

        for _, eleccion in elecciones_periodo.iterrows():
            fecha_elec = eleccion['fecha']
            if fecha_elec >= x.min() and fecha_elec <= x.max():
                y_max = ax.get_ylim()[1]

                arrow = FancyArrowPatch(
                    (fecha_elec, y_max * 0.88),
                    (fecha_elec, y_max * 0.3),
                    arrowstyle='->',
                    mutation_scale=40,
                    linewidth=4,
                    color='#CC0000',
                    zorder=10
                )
                ax.add_patch(arrow)

                ax.text(fecha_elec, y_max * 0.95, 'ELECCIÓN',
                       ha='center', fontsize=12, fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.5',
                                facecolor='white',
                                edgecolor='red',
                                linewidth=2))

        if alertas is not None and len(alertas) > 0:
            print(f"  📍 Mostrando {len(alertas)} alertas en el gráfico")
            for _, alerta in alertas.iterrows():
                if alerta['fecha_contrato'] >= x.min() and alerta['fecha_contrato'] <= x.max():
                    fecha_inicio_alerta = min(alerta['fecha_donacion'], alerta['fecha_contrato'])
                    fecha_fin_alerta = max(alerta['fecha_donacion'], alerta['fecha_contrato'])

                    ax.axvspan(fecha_inicio_alerta, fecha_fin_alerta,
                              alpha=0.15, color='yellow', zorder=1,
                              linewidth=2, edgecolor='orange', linestyle='--')

        ax.set_title(f'{partido.title()}',
                    fontsize=26, fontweight='bold', pad=20)
        ax.set_xlabel('Período', fontsize=14, fontweight='bold')
        ax.set_ylabel('Cantidad de Contratos\n(Donantes del Partido)',
                     fontsize=14, fontweight='bold')

        ax.xaxis.set_major_locator(YearLocator())
        ax.xaxis.set_major_formatter(DateFormatter('%Y'))

        ax.grid(axis='y', alpha=0.3, linestyle='--', linewidth=0.5)
        ax.set_axisbelow(True)

        ax.set_facecolor('#F5F5F5')
        fig.patch.set_facecolor('white')

        total_contratos = len(contratos_donantes)
        donantes_unicos = len(cedulas_partido)
        texto_stats = f"Contratos: {total_contratos:,} | Donantes: {donantes_unicos:,}"

        ax.text(0.02, 0.98, texto_stats,
               transform=ax.transAxes,
               fontsize=10,
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        plt.tight_layout()

        return fig, ax

    def generar_reporte_completo(self, ventana_alerta_meses=6, partido=None):
      """
      Genera un reporte completo con todas las métricas y análisis

      Parámetros:
      -----------
      ventana_alerta_meses : int, ventana temporal para detectar alertas (default: 6)
      partido : str, nombre del partido político para filtrar (opcional)
      """
      print("\n" + "=" * 90)
      if partido:
          print(f"REPORTE: ANÁLISIS DE CONTRATOS POST-ELECTORALES - {partido.upper()}")
      else:
          print("REPORTE COMPLETO: ANÁLISIS DE CONTRATOS POST-ELECTORALES")
      print("=" * 90)

      print(f"\n{'🚨 ALERTAS TEMPORALES':^90}")
      print("-" * 90)
      alertas = self.detectar_alertas_temporales(ventana_alerta_meses)

      # Filtrar por partido si se especificó
      if partido and len(alertas) > 0:
          partido_upper = partido.upper()
          alertas = alertas[alertas['partido_donado'].str.upper() == partido_upper]
          print(f"Filtrado por partido: {partido.upper()}")

      if len(alertas) > 0:
          print(f"\nTotal de alertas detectadas: {len(alertas)}")

          # Solo mostrar distribución por partido si NO se filtró
          if not partido:
              print(f"\n{'Alertas por partido:':^90}")
              alertas_por_partido = alertas['partido_donado'].value_counts()
              for p, cantidad in alertas_por_partido.head(10).items():
                  print(f"  • {p}: {cantidad} alertas")

          # Mostrar TODAS las alertas si se filtró por partido, sino top 10
          num_alertas_mostrar = len(alertas) if partido else 10
          titulo = f"Todas las alertas encontradas ({len(alertas)})" if partido else "Top 10 casos más sospechosos"

          print(f"\n{titulo:^90}")
          print("-" * 90)

          top_alertas = alertas.nsmallest(num_alertas_mostrar, 'diferencia_dias')

          for idx, (_, alerta) in enumerate(top_alertas.iterrows(), 1):
              print(f"\n{idx}. Cédula: {alerta['cedula']}")
              if 'nombre_contribuyente' in alerta and pd.notna(alerta['nombre_contribuyente']):
                  print(f"   Nombre: {alerta['nombre_contribuyente']}")
              print(f"   Partido: {alerta['partido_donado']}")
              print(f"   Donación: {alerta['fecha_donacion'].strftime('%Y-%m-%d')} "
                    f"(₡{alerta['monto_donacion']:,.0f})")
              print(f"   Contrato: {alerta['fecha_contrato'].strftime('%Y-%m-%d')}")
              if 'identificador_contrato' in alerta and pd.notna(alerta['identificador_contrato']):
                  print(f"   ID: {alerta['identificador_contrato']}")
              print(f"   Diferencia: {int(alerta['diferencia_dias'])} días "
                    f"({alerta['diferencia_meses']:.1f} meses)")
              print(f"   Secuencia: {'Donación → Contrato ⚠️' if alerta['donacion_antes'] else 'Contrato → Donación'}")
      else:
          if partido:
              print(f"\n✅ No se detectaron alertas para {partido.upper()}")
          else:
              print("\n✅ No se detectaron alertas")

      # Resumen de coincidencias (solo si no se filtró)
      if not partido:
          print(f"\n\n{'🔍 RESUMEN DE COINCIDENCIAS':^90}")
          print("-" * 90)
          print(f"\nTotal de cédulas que son DONANTES y PROVEEDORES: {len(self.coincidencias)}")

          contratos_coincidencias = self.df_contratos[
              self.df_contratos['cedula_proveedor'].isin(self.coincidencias)
          ]
          print(f"Contratos otorgados a estos proveedores: {len(contratos_coincidencias):,}")

      print("\n" + "=" * 90)

      return alerta

    def exportar_alertas_excel(self, alertas, ruta_archivo='alertas_temporales.xlsx'):
        """Exporta las alertas a un archivo Excel"""
        if len(alertas) == 0:
            print("⚠️  No hay alertas para exportar")
            return None

        alertas_ordenadas = alertas.sort_values('diferencia_dias')
        alertas_ordenadas.to_excel(ruta_archivo, index=False)
        print(f"✅ Alertas exportadas a: {ruta_archivo}")

        return ruta_archivo


print("✅ Sistema de Análisis V2 - COMPLETO")
print("   • Detección automática de columnas")
print("   • Filtro por partido político")
print("   • Gráficos tamaño (10, 5) por defecto")
print("   • Todas las funcionalidades implementadas")

✅ Sistema de Análisis V2 - COMPLETO
   • Detección automática de columnas
   • Filtro por partido político
   • Gráficos tamaño (10, 5) por defecto
   • Todas las funcionalidades implementadas


## 5️⃣ Inicializar Analizador

In [ ]:
# Crear instancia del analizador con tus datos
analizador = AnalizadorContratosElectorales(contratos, aportaciones)

📊 Preparando datos para análisis...

🔍 Analizando estructura de datos...

Contratos - columnas encontradas:
  1. Nro Contrato
  2. Identificador
  3. Secuencia
  4. Cédula Proveedor
  5. Nro Sicop
  6. Nro Procedimiento
  7. Ident Contrato Padre
  8. Contrato Modificado
  9. Tipo Modificacion
  10. Fecha Notificación
  ... y 4 más

Aportaciones - columnas encontradas:
  1. TIPO CONTRIBUCIÓN
  2. TIPO PERSONA
  3. PARTIDO POLÍTICO
  4. FECHA
  5. CÉDULA
  6. NOMBRE DEL CONTRIBUYENTE
  7. MONTO

🔧 Preparando contratos...
  ✓ Fecha: 'Fecha Notificación'
  ✓ Cédula Proveedor: 'Cédula Proveedor'
  ✓ Identificador: 'Identificador'
  ✓ Tipo Modificación: 'Tipo Modificacion'

🔧 Preparando donaciones...
  ✓ Partido: 'PARTIDO POLÍTICO'
  ✓ Fecha: 'FECHA'
  ✓ Cédula: 'CÉDULA'
  ✓ Monto: 'MONTO'
  ✓ Nombre: 'NOMBRE DEL CONTRIBUYENTE'
  ⚠️  Eliminadas 1 filas con fecha inválida

✅ Contratos procesados: 396,676
✅ Donaciones procesadas: 97,009
✅ Proveedores únicos: 16,783
✅ Donantes únicos: 29,123

🔍

## 6️⃣ Generar Reporte Completo

In [ ]:
# Generar reporte con ventana de 6 meses
alertas = analizador.generar_reporte_completo(ventana_alerta_meses=6, partido='ACCION CIUDADANA')


REPORTE: ANÁLISIS DE CONTRATOS POST-ELECTORALES - ACCION CIUDADANA

                                   🚨 ALERTAS TEMPORALES                                   
------------------------------------------------------------------------------------------

🔎 Detectando alertas temporales (ventana: 6 meses)...
⚠️  Total de alertas detectadas: 300
Filtrado por partido: ACCION CIUDADANA

Total de alertas detectadas: 62

                            Todas las alertas encontradas (62)                            
------------------------------------------------------------------------------------------

1. Cédula: 204090052
   Nombre: CARRANZA CASCANTE LUIS RAMON
   Partido: ACCION CIUDADANA
   Donación: 2017-11-01 (₡35,428)
   Contrato: 2017-11-01
   ID: CE201711000031
   Diferencia: 0 días (0.0 meses)
   Secuencia: Donación → Contrato ⚠️

2. Cédula: 206230945
   Nombre: ARGUELLO MIRANDA ELIZABETH DEL CARMEN
   Partido: ACCION CIUDADANA
   Donación: 2021-11-01 (₡128,368)
   Contrato: 2021-11-04
 

## 7️⃣ Explorar Alertas Detectadas

In [ ]:
# Ver las alertas en formato tabla
if len(alertas) > 0:
    print(f"Total de alertas: {len(alertas)}\n")

    # Mostrar primeras alertas
    display(alertas.head(10))

    # Distribución por partido
    print("\n\nAlertas por partido:")
    display(alertas['partido_donado'].value_counts())

    # Estadísticas básicas
    print("\n\nEstadísticas de diferencias temporales:")
    print(f"Diferencia mínima: {alertas['diferencia_dias'].min():.0f} días")
    print(f"Diferencia máxima: {alertas['diferencia_dias'].max():.0f} días")
    print(f"Diferencia promedio: {alertas['diferencia_dias'].mean():.1f} días")
    print(f"Diferencia mediana: {alertas['diferencia_dias'].median():.1f} días")
else:
    print("No se detectaron alertas en el período")

Total de alertas: 300



,cedula,fecha_donacion,fecha_contrato,partido_donado,monto_donacion,diferencia_dias,diferencia_meses,donacion_antes,año_donacion,año_contrato,nombre_contribuyente,identificador_contrato,tipo_modificacion
0,401050890,2020-01-28,2020-05-20 13:51:49,SENTIR HEREDIA,20000.0,113,3.712221,True,2020,2020,VARGAS BENAVIDES WALTER ENRIQUE DE JESUS,CE202003002015,NaN
1,114460813,2020-01-31,2020-03-26 09:27:26,ACCION CIUDADANA,53920.0,55,1.806833,True,2020,2020,VALLEJO RIVAS ALEJANDRO YAVE,CE202003001851,NaN
2,110940767,2023-11-22,2023-09-29 15:39:49,BIENESTAR RAFAELE?O,12500.0,54,1.773982,False,2023,2023,CALVO ARROYO KAROL MARIA,CE202307000305,Modificación unilateral del contrato (Aumento)
3,110940767,2023-11-22,2023-09-27 10:47:31,BIENESTAR RAFAELE?O,12500.0,56,1.839685,False,2023,2023,CALVO ARROYO KAROL MARIA,CE202307000305,Modificación unilateral del contrato (Aumento)
4,110940767,2023-11-22,2023-08-21 08:50:06,BIENESTAR RAFAELE?O,12500.0,93,3.055191,False,2023,2023,CALVO ARROYO KAROL MARIA,CE202205001959,Modificación unilateral del contrato (Aumento)
5,110940767,2023-11-22,2023-08-10 15:19:45,BIENESTAR RAFAELE?O,12500.0,104,3.416557,False,2023,2023,CALVO ARROYO KAROL MARIA,CE202202001359,Prórrogas al contrato
6,110940767,2023-11-22,2023-07-18 08:38:33,BIENESTAR RAFAELE?O,12500.0,127,4.172142,False,2023,2023,CALVO ARROYO KAROL MARIA,CE202307000305,NaN
7,110940767,2023-11-22,2024-03-06 11:06:52,BIENESTAR RAFAELE?O,12500.0,105,3.449409,True,2023,2024,CALVO ARROYO KAROL MARIA,CE202403000249,NaN
8,110940767,2023-11-22,2024-02-08 14:32:59,BIENESTAR RAFAELE?O,12500.0,78,2.562418,True,2023,2024,CALVO ARROYO KAROL MARIA,CE202307000305,Modificación de otras cláusulas
9,110940767,2023-11-22,2024-02-08 14:30:17,BIENESTAR RAFAELE?O,12500.0,78,2.562418,True,2023,2024,CALVO ARROYO KAROL MARIA,CE202205001959,Modificación de otras cláusulas




Alertas por partido:


,count
partido_donado,
ACCION CIUDADANA,62
CURRIDABAT SIGLO XXI,57
SENTIR HEREDIA,40
AGENDA DEMOCRATICA NACIONAL,32
BIENESTAR RAFAELE?O,27
LIBERAL PROGRESISTA,16
FRENTE AMPLIO,13
PROGRESO SOCIAL DEMOCRATICO,11
EN COMUN,10




Estadísticas de diferencias temporales:
Diferencia mínima: 0 días
Diferencia máxima: 182 días
Diferencia promedio: 80.9 días
Diferencia mediana: 74.0 días


## 8️⃣ Exportar Alertas a Excel

In [ ]:
# Exportar alertas a tu Google Drive
if len(alertas) > 0:
    ruta_exportacion = '/content/drive/MyDrive/Concurso/alertas_6meses.xlsx'
    analizador.exportar_alertas_excel(alertas, ruta_exportacion)
    print(f"\n✅ Archivo guardado en: {ruta_exportacion}")

✅ Alertas exportadas a: /content/drive/MyDrive/Concurso/alertas_6meses.xlsx

✅ Archivo guardado en: /content/drive/MyDrive/Concurso/alertas_6meses.xlsx
